# Compressed Array Example

This notebook demonstrates how to use the array utilities for compressing numpy arrays.

We'll cover:
1. Creating numpy arrays of different shapes and types
2. Compressing the arrays using meshoptimizer's encoding
3. Saving the compressed arrays to files and zip archives
4. Loading the compressed arrays from files and zip archives
5. Working with multiple arrays in a single zip file

In [1]:
import os
import numpy as np
from meshoptimizer import (
    encode_array,
    decode_array,
    save_array_to_file,
    load_array_from_file,
    save_array_to_zip,
    load_array_from_zip,
    save_arrays_to_zip,
    load_arrays_from_zip
)

## Creating Example Arrays

Let's create some example arrays of different shapes and types:

In [2]:
# 1D array of float32 values
array_1d = np.linspace(0, 10, 1000, dtype=np.float32)
print(f"1D array: shape={array_1d.shape}, dtype={array_1d.dtype}, size={array_1d.nbytes} bytes")

# 2D array of float32 values (e.g., vertex positions)
array_2d = np.random.random((1000, 3)).astype(np.float32)
print(f"2D array: shape={array_2d.shape}, dtype={array_2d.dtype}, size={array_2d.nbytes} bytes")

# 3D array of float32 values (e.g., voxel data)
array_3d = np.random.random((20, 20, 20)).astype(np.float32)
print(f"3D array: shape={array_3d.shape}, dtype={array_3d.dtype}, size={array_3d.nbytes} bytes")

# 2D array of int32 values
array_int = np.random.randint(0, 100, (100, 100), dtype=np.int32)
print(f"Integer array: shape={array_int.shape}, dtype={array_int.dtype}, size={array_int.nbytes} bytes")

1D array: shape=(1000,), dtype=float32, size=4000 bytes
2D array: shape=(1000, 3), dtype=float32, size=12000 bytes
3D array: shape=(20, 20, 20), dtype=float32, size=32000 bytes
Integer array: shape=(100, 100), dtype=int32, size=40000 bytes


## Example 1: Basic Encoding and Decoding

Let's encode and decode a 2D array:

In [3]:
# Encode the 2D array
encoded_array = encode_array(array_2d)
print(f"Encoded size: {len(encoded_array.data)} bytes")
print(f"Compression ratio: {len(encoded_array.data) / array_2d.nbytes:.2f}")

# Decode the array
decoded_array = decode_array(encoded_array)
print(f"Decoded array: shape={decoded_array.shape}, dtype={decoded_array.dtype}")

# Verify the decoded array matches the original
is_close = np.allclose(decoded_array, array_2d, rtol=1e-5)
print(f"Decoded array matches original: {is_close}")

Encoded size: 10414 bytes
Compression ratio: 0.87
Decoded array: shape=(1000, 3), dtype=float32
Decoded array matches original: True


## Example 2: Saving and Loading from a File

Now let's save the array to a file and load it back:

In [4]:
# Save the array to a file
file_path = "compressed_array.bin"
save_array_to_file(array_2d, file_path)
print(f"Saved to file: {file_path}")
print(f"File size: {os.path.getsize(file_path)} bytes")
print(f"Compression ratio: {os.path.getsize(file_path) / array_2d.nbytes:.2f}")

# Load the array from the file
loaded_array = load_array_from_file(file_path)
print(f"Loaded array: shape={loaded_array.shape}, dtype={loaded_array.dtype}")

# Verify the loaded array matches the original
is_close = np.allclose(loaded_array, array_2d, rtol=1e-5)
print(f"Loaded array matches original: {is_close}")

Saved to file: compressed_array.bin
File size: 10473 bytes
Compression ratio: 0.87
Loaded array: shape=(1000, 3), dtype=float32
Loaded array matches original: True


## Example 3: Saving and Loading from a Zip File

Let's save the array to a zip file and load it back:

In [5]:
# Save the array to a zip file
zip_path = "compressed_array.zip"
save_array_to_zip(array_2d, zip_path)
print(f"Saved to zip file: {zip_path}")
print(f"Zip file size: {os.path.getsize(zip_path)} bytes")
print(f"Compression ratio: {os.path.getsize(zip_path) / array_2d.nbytes:.2f}")

# Load the array from the zip file
loaded_array = load_array_from_zip(zip_path)
print(f"Loaded array: shape={loaded_array.shape}, dtype={loaded_array.dtype}")

# Verify the loaded array matches the original
is_close = np.allclose(loaded_array, array_2d, rtol=1e-5)
print(f"Loaded array matches original: {is_close}")

Saved to zip file: compressed_array.zip
Zip file size: 10604 bytes
Compression ratio: 0.88
Loaded array: shape=(1000, 3), dtype=float32
Loaded array matches original: True


## Example 4: Working with Multiple Arrays

Let's save multiple arrays to a single zip file and load them back:

In [6]:
# Create a dictionary of arrays
arrays = {
    "array_1d": array_1d,
    "array_2d": array_2d,
    "array_3d": array_3d,
    "array_int": array_int
}

# Save all arrays to a single zip file
multi_zip_path = "multiple_arrays.zip"
save_arrays_to_zip(arrays, multi_zip_path)
print(f"Saved multiple arrays to zip file: {multi_zip_path}")

# Calculate total size of all arrays
total_size = sum(array.nbytes for array in arrays.values())
print(f"Total size of all arrays: {total_size} bytes")
print(f"Zip file size: {os.path.getsize(multi_zip_path)} bytes")
print(f"Compression ratio: {os.path.getsize(multi_zip_path) / total_size:.2f}")

# Load all arrays from the zip file
loaded_arrays = load_arrays_from_zip(multi_zip_path)
print(f"Loaded {len(loaded_arrays)} arrays from zip file")

# Verify each loaded array matches the original
for name, array in arrays.items():
    is_close = np.allclose(loaded_arrays[name], array, rtol=1e-5)
    print(f"Array '{name}' matches original: {is_close}")

Saved multiple arrays to zip file: multiple_arrays.zip
Total size of all arrays: 88000 bytes
Zip file size: 50988 bytes
Compression ratio: 0.58
Loaded 4 arrays from zip file
Array 'array_1d' matches original: True
Array 'array_2d' matches original: True
Array 'array_3d' matches original: True
Array 'array_int' matches original: True


## Cleaning Up

Let's clean up the files we created:

In [7]:
# Clean up
for path in [file_path, zip_path, multi_zip_path]:
    if os.path.exists(path):
        os.remove(path)
        print(f"Removed {path}")

print("\nExample completed successfully!")

Removed compressed_array.bin
Removed compressed_array.zip
Removed multiple_arrays.zip

Example completed successfully!
